In [8]:
import pandas as pd # pip install pandas
import re # pip install re
import string 
import markdown # pip install markdown
from bs4 import BeautifulSoup # pip install beautifulsoup4
import csv
import emoji
import nltk

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

hot = pd.read_csv("../dataset/raw/hot.csv")
top = pd.read_csv("../dataset/raw/top.csv")
controversial = pd.read_csv("../dataset/raw/controversial.csv")
new = pd.read_csv("../dataset/raw/new.csv")

df = pd.concat([hot, top, controversial, new], ignore_index=True)

In [9]:
df.dropna(subset=['Content'], inplace=True)
df.shape

(1978, 10)

In [10]:
import unicodedata
import sys
URL_REGEX = r".*(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*).*"
punctuation = "".join((chr(i) for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')))

# Remove URLs
def remove_url(cell):
    return re.sub(URL_REGEX, "", str(cell))

# Convert markdown to plaintext 
def md_to_text(cell):
    html = markdown.markdown(cell)
    soup = BeautifulSoup(html, features='html.parser')
    return soup.get_text()

# Replace NaN and \n 
def remove_NaN_newline(cell):
    return re.sub(r"(\n|NaN|nan)", " ", str(cell))

def remove_punctuation(cell):
    return cell.translate(str.maketrans('', '', punctuation))

# Convert slangs
def replace_abbrv(cell):
    user_string = str(cell)
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        fileName = "slang.txt"
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # if re.findall(fr"{_str}", row[0], flags=re.IGNORECASE):
                if _str.lower() == row[0].lower():
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    return ' '.join(user_string)

def remove_emoji(text):
    return emoji.replace_emoji(text, replace="")

def remove_numbers(cell):
    return re.sub(r"\d+", " ", str(cell))

def lowercase(cell):
    return str(cell).lower()

In [4]:
df['Content'] = df['Content'].apply(replace_abbrv)
df['Content'] = df['Content'].apply(remove_url)
df['Content'] = df['Content'].apply(md_to_text)
df['Content'] = df['Content'].apply(remove_NaN_newline)
df['Content'] = df['Content'].apply(remove_emoji)
df['Content'] = df['Content'].apply(remove_numbers)
df['Content'] = df['Content'].apply(lowercase)
df['Content'] = df['Content'].apply(remove_punctuation)

df['Title'] = df['Title'].apply(replace_abbrv)
df['Title'] = df['Title'].apply(remove_url)
df['Title'] = df['Title'].apply(md_to_text)
df['Title'] = df['Title'].apply(remove_NaN_newline)
df['Title'] = df['Title'].apply(remove_emoji)
df['Title'] = df['Title'].apply(remove_numbers)
df['Title'] = df['Title'].apply(lowercase)
df['Title'] = df['Title'].apply(remove_punctuation)

KeyboardInterrupt: 

In [11]:
df.insert(4, "Title+Content", df["Title"] + " " + df["Content"])
df.reset_index(drop=True, inplace=True)

### Translation to English

In [ ]:
from deep_translator import GoogleTranslator
import math

In [ ]:
def trans(text):
    if len(text) > 5000:
        text_1 = GoogleTranslator(source='tl', target='en').translate(text[:math.floor(len(text)/2)]) 
        text_2 = GoogleTranslator(source='tl', target='en').translate(text[math.floor(len(text)/2):]) 
        return text_1 + " " + text_2
    else:   
        return GoogleTranslator(source='tl', target='en').translate(text)

In [ ]:
trans_1 = df['Title+Content'][:500].apply(trans)

In [ ]:
trans_2 = df['Title+Content'][500:1000].apply(trans)

In [ ]:
trans_3 = df['Title+Content'][1000:1500].apply(trans)

In [ ]:
trans_4 = df['Title+Content'][1500:2000].apply(trans)

In [ ]:
trans_5 = df['Title+Content'][2000:].apply(trans)

In [ ]:
df_test = pd.concat([trans_1, trans_2, trans_3, trans_4, trans_5])

In [ ]:
df_test = df_test.to_frame().rename(columns={"Title+Content":"Translated"})

In [ ]:
df_test["Translated"] = df_test["Translated"].apply(remove_punctuation)
df_test["Translated"] = df_test["Translated"].apply(lowercase)

In [ ]:
df.insert(5, "Translated", df_test)

In [ ]:
df.head(5)

,Timestamp,Content Type,Title,Content,Upvotes Count,Translated,Comments Count,Upvote Ratio,Flair,Permalink,Reddit ID,Title+Content
0,2024-04-21 13:01:23,Text-only,esena,help i filed a request for assistance sa esena re last pay na inabot ng months at coe last friday meron na akong ref number ng request for assistance kaso that same day biglang nagmessage ang human resources na okay na ang cheke ng last pay ko at pwede ko na sya ipick up my question is paano icancel yung request for assistance since nakuha ko na yung last pay at coe ko thank you po,2,esena help i filed a request for assistance at esena re last pay that took months and coe last friday i have the ref number of the request for assistance case that same day suddenly the human resources messaged that the check of my last pay is okay and can i have already picked him up my question is how to cancel the request for assistance since i already got my last pay and coe thank you,3,1.00,Advice Needed 🤔,https://www.reddit.com//r/AntiworkPH/comments/1c99rt3/esena/,1c99rt3,esena help i filed a request for assistance sa esena re last pay na inabot ng months at coe last friday meron na akong ref number ng request for assistance kaso that same day biglang nagmessage ang human resources na okay na ang cheke ng last pay ko at pwede ko na sya ipick up my question is paano icancel yung request for assistance since nakuha ko na yung last pay at coe ko thank you po
1,2024-04-20 14:50:31,Text-only,dodged a bad company and human resources i guess,ive applied for one of the subsidiaries of aboitiz and id really like to share how horrible the application process or how the human resources handled it i guess was bale i just graduated from my electrical engineering degree this january and ive started to apply to a few companies including this subsidiary from aboitiz when compared to companies ive also done interviews with like pldt jg summit and smc sobrang nakakadisappoint yung experience ko with aboitiz i had my expectations for aboitiz since it was known to be one of the top companies and employers sa philippines kaya siguro naman formal and may class talaga yung way kung paano makipagcommunicate yung human resources right kaso hindi para bang nakikipagcommunicate lang ako with someone sa isang lowend company who is deeply struggling in handling communications ang problema pa rito was that i was the one who was understanding and patient kahit na sila yung may lapses in handling my application three months have passed and theyve contacted me saying that my application was already being considered but it was placed on hold due to a problem sa vacancy ng position with that i just had to withdraw and say that i wasnt already comfortable with the process and how poorly they were handling my application i withdrew kasi baka my application was a hassle on their end pala tsaka baka hinihintay lang talaga ng human resources na magbreak ako towards their treatmentattitude nagreply yung human resources sa withdrawal ko hindi ko malagay exact phrases but heres the gist human resources said the usual stuff muna like were deeply sorry about the inconviences made on your end blablabla sorry to see you go tapos ayon sa dulo sinabi na sobrang arte ko raw for complaining and that i should man up more to understand reallife problems that are being encounted sa office nila i shouldnt be complaining din daw kasi hindi naman ako employee to ask for something on their end what i was just asking for was respect heres one one out of a lot inconvenienceproblem that happened they asked me to attend an f f interview sa office nila the two hour travel going to their office didnt matter kasi i was enjoying my stay sa province namin to avoid the city heat however when i arrived then waited for two hours sa lobby nobody sa division division ata or department kung saan yung mga mamemeet ko remembered that i had a scheduled interview the human resources responsible for this was called to have a meeting sa ibang office at

In [ ]:
df.to_csv(f"../dataset/_compiled/Compiled.csv", index=False)

In [ ]:
# df_save = df.copy(deep=True)